In [1]:
# Basic
import sys
import numpy as np
from datetime import datetime

# Keras Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score

# TT Layer
from TTLayer import TT_Layer

# misc
from keras.utils.np_utils import to_categorical


np.random.seed(11111986)

In [2]:
# choose one data set:
#data_name = 'prostate'
data_name = 'lymphoma'
# data_name = 'mice'
# data_name = 'yeast'

data_path = './Datasets/GenExpress/'
X_train = np.loadtxt(data_path + data_name + '/' + data_name + '_train.data.gz')
Y_train = np.loadtxt(data_path + data_name + '/' + data_name + '_train.labels.gz')
X_valid = np.loadtxt(data_path + data_name + '/' + data_name + '_valid.data.gz')
Y_valid = np.loadtxt(data_path + data_name + '/' + data_name + '_valid.labels.gz')


n, d = X_train.shape
print('Training data has shape = ' + str(X_train.shape))
print('Valid data has shape = ' + str(X_valid.shape))

# Additional columns so that the TT can factorize the feature dimension
new_dim = None
if data_name == 'prostate':
    new_dim = 6300
elif data_name == 'lymphoma':
    new_dim = 4096
elif data_name == 'mice':
    new_dim = 150
elif data_name == 'yeast':
    new_dim = 120

Training data has shape = (50, 4026)
Valid data has shape = (12, 4026)


In [3]:
X_train = np.concatenate([X_train, np.zeros((X_train.shape[0], new_dim - d))], axis=1)
col_shuffle = np.random.choice(range(new_dim), new_dim, False)
X_train = X_train[:, col_shuffle]
X_valid = np.concatenate([X_valid, np.zeros((X_valid.shape[0], new_dim - d))], axis=1)
X_valid = X_valid[:, col_shuffle]

In [4]:
X_train

array([[ 0.87685906,  0.0027742 ,  0.        , ..., -0.69961542,
         0.47103395,  0.22129542],
       [ 0.00515171, -0.12848889,  0.        , ...,  1.64224898,
         0.84040542,  0.32811648],
       [-0.92978768,  1.33690788,  0.        , ...,  0.31076867,
        -0.76131706,  0.11166704],
       ...,
       [-0.56006992,  0.43160397,  0.        , ...,  0.90325375,
        -0.54797633,  0.40741681],
       [ 0.08428151,  0.35179755,  0.        , ..., -1.1125008 ,
        -0.29587287,  0.16876026],
       [-1.12444027, -1.1831339 ,  0.        , ...,  0.65593313,
         0.7341913 , -0.57663307]])

In [5]:
normalization = 1
if normalization == 0:
    X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
    X_train[np.where(np.isnan(X_train))] = 0.
    X_valid = (X_valid - X_valid.mean(axis=0)) / X_valid.std(axis=0)
    X_valid[np.where(np.isnan(X_valid))] = 0.
elif normalization == 1:
    X_train = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
    X_valid = (X_valid - X_valid.min(axis=0)) / (X_valid.max(axis=0) - X_valid.min(axis=0))
    X_train[np.where(np.isnan(X_train))] = 0.
    X_valid[np.where(np.isnan(X_valid))] = 0.

/tmp/ipykernel_3543663/1832389541.py:8: RuntimeWarning: invalid value encountered in true_divide
  X_train = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
/tmp/ipykernel_3543663/1832389541.py:9: RuntimeWarning: invalid value encountered in true_divide
  X_valid = (X_valid - X_valid.min(axis=0)) / (X_valid.max(axis=0) - X_valid.min(axis=0))


In [6]:
Y_train

array([0., 2., 0., 1., 0., 0., 0., 2., 0., 0., 2., 2., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 0., 1., 0., 2., 2., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 2., 2., 0., 0., 0., 0., 2., 0., 0.])

In [7]:
if data_name == 'lymphoma':
    Y_train = to_categorical(Y_train.astype('int32'))
    Y_valid = to_categorical(Y_valid.astype('int32'))

# Hyper-parameters
if data_name == 'prostate':
    alpha = 0.1
    tt_alpha = 5e-4
    nb_epoch = 200
    batch_size = 10
    lr = 1e-4
    tt_input_shape = [7, 9, 10, 10]
    tt_output_shape = [5, 5, 5, 5]
    tt_ranks = [1, 15, 15, 15, 1]
elif data_name == 'lymphoma':
    alpha = 0.1
    tt_alpha = 5e-4
    nb_epoch = 200
    batch_size = 10
    lr = 1e-4
    tt_input_shape = [8, 8, 8, 8]
    tt_output_shape = [5, 5, 5, 5]
    tt_ranks = [1, 15, 15, 15, 1]
elif data_name == 'mice':
    alpha = 0.1
    tt_alpha = 5e-4
    nb_epoch = 100
    batch_size = 7
    lr = 1e-4
    tt_input_shape = [5, 6, 5]
    tt_output_shape = [15, 15, 15]
    tt_ranks = [1, 10, 10, 1]
elif data_name == 'yeast':
    alpha = 0.001
    tt_alpha = 0
    nb_epoch = 500
    batch_size = 5
    lr = 1e-4
    tt_input_shape = [4, 5, 6]
    tt_output_shape = [5, 5, 5]
    tt_ranks = [1, 10, 10, 1]

In [8]:
# Model with fully connected layer
if data_name == 'prostate':
    input = Input(shape=(new_dim,))
    h = Dense(np.prod(tt_output_shape), activation='sigmoid', kernel_regularizer=l2(alpha))(input)
    output = Dense(1, activation='sigmoid', kernel_regularizer=l2(alpha))(h)
    model_full = Model(input, output)
    model_full.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])
elif data_name == 'lymphoma':
    input = Input(shape=(new_dim,))
    h = Dense(np.prod(tt_output_shape), activation='sigmoid', kernel_regularizer=l2(alpha))(input)
    output = Dense(Y_train.shape[1], activation='softmax', kernel_regularizer=l2(alpha))(h)
    model_full = Model(input, output)
    model_full.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
elif data_name in ['mice', 'yeast']:
    input = Input(shape=(new_dim,))
    h = Dense(np.prod(tt_output_shape), activation='sigmoid', kernel_regularizer=l2(alpha))(input)
    output = Dense(Y_train.shape[1], activation='linear', kernel_regularizer=l2(alpha))(h)
    model_full = Model(input, output)
    model_full.compile(optimizer=Adam(lr), loss='mse')

start_full = datetime.now()
for l in range(nb_epoch):
    if_print = l % 10 == 0
    if if_print:
        print('iter = ' + str(l))
        verbose = 2
    else:
        verbose = 0
    history = model_full.fit(x=X_train, y=Y_train, verbose=verbose, epochs=1, batch_size=batch_size,
                             validation_split=0.2)
stop_full = datetime.now()

2022-09-15 03:40:06.079810: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-15 03:40:06.113619: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


iter = 0
4/4 - 2s - loss: 108.9337 - accuracy: 0.5250 - val_loss: 106.8284 - val_accuracy: 0.7000 - 2s/epoch - 501ms/step
iter = 10
4/4 - 0s - loss: 78.7626 - accuracy: 0.9750 - val_loss: 77.1241 - val_accuracy: 1.0000 - 137ms/epoch - 34ms/step
iter = 20
4/4 - 0s - loss: 55.7075 - accuracy: 1.0000 - val_loss: 54.5288 - val_accuracy: 1.0000 - 158ms/epoch - 39ms/step
iter = 30
4/4 - 0s - loss: 38.9089 - accuracy: 1.0000 - val_loss: 38.0660 - val_accuracy: 1.0000 - 137ms/epoch - 34ms/step
iter = 40
4/4 - 0s - loss: 26.8659 - accuracy: 1.0000 - val_loss: 26.2937 - val_accuracy: 1.0000 - 217ms/epoch - 54ms/step
iter = 50
4/4 - 0s - loss: 18.3572 - accuracy: 1.0000 - val_loss: 17.9449 - val_accuracy: 1.0000 - 194ms/epoch - 48ms/step
iter = 60
4/4 - 0s - loss: 12.4237 - accuracy: 1.0000 - val_loss: 12.1414 - val_accuracy: 1.0000 - 194ms/epoch - 48ms/step
iter = 70
4/4 - 0s - loss: 8.3561 - accuracy: 1.0000 - val_loss: 8.1823 - val_accuracy: 1.0000 - 152ms/epoch - 38ms/step
iter = 80
4/4 - 0s 

In [9]:
# Model with TT layer
from keras.layers import Dropout
if data_name == 'prostate':
    input_TT = Input(shape=(new_dim,))
    tt = TT_Layer(tt_input_shape, tt_output_shape, kernel_regularizer=l2(tt_alpha), tt_ranks=tt_ranks, activation='sigmoid')
    h_TT = tt(input_TT)
    h_TT = Dropout(0.5)(h_TT)
    output_TT = Dense(1, activation='sigmoid', kernel_regularizer=l2(alpha))(h_TT)
    model_TT = Model(input_TT, output_TT)
    model_TT.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])
elif data_name == 'lymphoma':
    input_TT = Input(shape=(new_dim,))
    tt = TT_Layer(tt_input_shape, tt_output_shape, kernel_regularizer=l2(tt_alpha),
                  tt_ranks=tt_ranks, activation='sigmoid')
    h_TT = tt(input_TT)
    output_TT = Dense(Y_train.shape[1], activation='softmax', kernel_regularizer=l2(alpha))(h_TT)
    model_TT = Model(input_TT, output_TT)
    model_TT.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
elif data_name in ['mice', 'yeast']:
    input_TT = Input(shape=(new_dim,))
    tt = TT_Layer(tt_input_shape=tt_input_shape, tt_output_shape=tt_output_shape, kernel_regularizer=l2(tt_alpha),
                  tt_ranks=tt_ranks, bias=True, activation='sigmoid', ortho_init=True)
    h_TT = tt(input_TT)
    output_TT = Dense(output_dim=Y_train.shape[1], activation='linear', kernel_regularizer=l2(alpha))(h_TT)
    model_TT = Model(input=input_TT, output=output_TT)
    model_TT.compile(optimizer=Adam(lr), loss='mse')


start_TT = datetime.now()
for l in range(nb_epoch):
    if_print = l % 10 == 0
    if if_print:
        print('iter = ' + str(l))
        verbose = 2
    else:
        verbose = 0
    history = model_TT.fit(x=X_train, y=Y_train, verbose=verbose, epochs=1, batch_size=batch_size,
                           validation_split=0.2)

stop_TT = datetime.now()

Compression factor = 19200 / 2560000 = 0.0075
iter = 0
4/4 - 1s - loss: 2.1185 - accuracy: 0.1500 - val_loss: 2.0534 - val_accuracy: 0.0000e+00 - 689ms/epoch - 172ms/step
iter = 10
4/4 - 0s - loss: 1.4484 - accuracy: 0.7000 - val_loss: 1.4478 - val_accuracy: 0.7000 - 208ms/epoch - 52ms/step
iter = 20
4/4 - 0s - loss: 1.4180 - accuracy: 0.7000 - val_loss: 1.4192 - val_accuracy: 0.7000 - 116ms/epoch - 29ms/step
iter = 30
4/4 - 0s - loss: 1.4074 - accuracy: 0.7000 - val_loss: 1.4034 - val_accuracy: 0.7000 - 64ms/epoch - 16ms/step
iter = 40
4/4 - 0s - loss: 1.3949 - accuracy: 0.7000 - val_loss: 1.3916 - val_accuracy: 0.7000 - 114ms/epoch - 29ms/step
iter = 50
4/4 - 0s - loss: 1.3839 - accuracy: 0.7000 - val_loss: 1.3828 - val_accuracy: 0.7000 - 132ms/epoch - 33ms/step
iter = 60
4/4 - 0s - loss: 1.3699 - accuracy: 0.7000 - val_loss: 1.3708 - val_accuracy: 0.7000 - 167ms/epoch - 42ms/step
iter = 70
4/4 - 0s - loss: 1.3572 - accuracy: 0.7000 - val_loss: 1.3549 - val_accuracy: 0.7000 - 85ms/ep

In [10]:
if data_name not in ['mice', 'yeast']:
    print('#######################################################')
    Y_pred_full =  model_full.predict(X_valid)
    print('Results of the model with fully connected layer')
    print('Time consumed: ' + str(stop_full - start_full))
    print('Accuracy: ' + str(accuracy_score(Y_valid, np.round(Y_pred_full))))
    print('AUROC: ' + str(roc_auc_score(Y_valid, Y_pred_full)))
    print('AUPRC: ' + str(average_precision_score(Y_valid, Y_pred_full)))
    print('#######################################################')
    Y_pred_TT = model_TT.predict(X_valid)
    print('Results of the model with TT layer')
    print('Time consumed: ' + str(stop_TT - start_TT))
    print('Parameter compression factor: ' + str(tt.compress_factor))
    print('Accuracy: ' + str(accuracy_score(Y_valid, np.round(Y_pred_TT))))
    print('AUROC: ' + str(roc_auc_score(Y_valid, Y_pred_TT)))
    print('AUPRC: ' + str(average_precision_score(Y_valid, Y_pred_TT)))
else:
    print('#######################################################') 
    print(Y_pred_full = model_full.predict(X_valid))
    print('Results of the model with fully connected layer')
    print('Time consumed: ' + str(stop_full - start_full))
    print('MSE: ' + str(((Y_valid - Y_pred_full)**2).mean()))

    print('#######################################################')
    Y_pred_TT = model_TT.predict(X_valid)
    print('Results of the model with TT layer')
    print('Time consumed: ' + str(stop_TT - start_TT))
    print('Parameter compression factor: ' + str(tt.compress_factor))
    print('MSE: ' + str(((Y_valid - Y_pred_TT)**2).mean()))

#######################################################
Results of the model with fully connected layer
Time consumed: 0:00:48.576510
Accuracy: 0.9166666666666666
AUROC: 1.0
AUPRC: 1.0
#######################################################
Results of the model with TT layer
Time consumed: 0:00:35.793167
Parameter compression factor: 0.0075
Accuracy: 0.5833333333333334
AUROC: 0.8292768959435626
AUPRC: 0.7338624338624338
